In [ ]:
base_dir = '/home/labs/hornsteinlab/Collaboration/FUNOVA/input/images/raw'
NOVA_HOME = '/home/labs/hornsteinlab/Collaboration/NOVA_GAL/NOVA'
FILE_EXTENSION = '.tiff'
KEY_BATCH = 'Batch'

In [ ]:
import os
import sys
os.environ['NOVA_HOME'] = NOVA_HOME
sys.path.insert(1, os.getenv('NOVA_HOME'))
from tools.show_images_utils import *
%load_ext autoreload
%autoreload 2

In [ ]:
df_processed = extract_processed_image_metadata('/home/labs/hornsteinlab/Collaboration/FUNOVA/input/images/processed', FILE_EXTENSION = '.npy', KEY_BATCH = KEY_BATCH)
df_processed

In [ ]:
df_raw = extract_image_metadata(base_dir, FILE_EXTENSION = FILE_EXTENSION, KEY_BATCH = KEY_BATCH)
df_raw

In [ ]:
# Rename columns in df_raw
df_raw = df_raw.rename(columns={
    'Path': 'Path_raw',
    'RootFolder': 'RootFolder_raw',
})

# Rename columns in df_processed
df_processed = df_processed.rename(columns={
    'Path': 'Path_processed',
    'RootFolder': 'RootFolder_processed',
})

# Define columns to match on (common metadata columns)
match_cols = ['Marker', 'Condition', 'CellLine', 'Batch_Rep', 'Rep', 'Batch', 'Panel', 'Image_Name']

# Drop duplicates to avoid exploding the merge if raw has multiple matches
df_raw_unique = df_raw.drop_duplicates(subset=match_cols)

# Merge df_processed with raw to get Path_raw, RootFolder_raw
df_combined = df_processed.merge(
    df_raw_unique[match_cols + ['Path_raw', 'RootFolder_raw']],
    on=match_cols,
    how='left'
)
df_combined


In [ ]:
for key in df_combined.iloc[5].keys():
    print(key,df_combined.iloc[5][key])

In [ ]:
for col in ['Marker', 'Condition', 'CellLine', 'Rep', 'Batch', 'Panel']:
    print(col, 'values are:', np.unique(df_combined[col]))

In [ ]:
# Example Usage:

# Show images for the marker 'P54'
# Filters: default parameters for all other options
# show_images(df, 'P54')

# Show images for the marker 'G3BP1'
# Exclude DAPI, limit to 3 samples, and filter by Batch 1, Rep 1, WT cell line, and Untreated condition
# show_images(df, marker='G3BP1', show_DAPI=False, samples=3, batch=1, rep=1, cell_line='WT', condition='Untreated')

# get df of specific images
# get_specific_imgs(df, marker='G3BP1', show_DAPI=False, samples=3, batch=1, rep=1, cell_line='WT', condition='Untreated')

# Create a PDF report for the marker 'TIA1' under the 'Untreated' condition
# Output the report as 'marker_name_Untreated/stress_Report.pdf'
# create_img_pdf_report(df, marker='TIA1', condition='Untreated', output_file='Untreated_Report.pdf')

In [ ]:
show_images(df, marker = 'DAPI', show_DAPI = False, batch = 2, cell_line ='C9orf72-HRE' , funova = True, samples = 10)

In [ ]:
for marker in df_combined['Marker'].unique():
    for condition in ['Untreated', 'stress']:
        create_img_pdf_report(df_combined, marker=marker, condition=condition, output_file=f'{condition}_Report.pdf',
                     reps=2, batches=4, path_key='Path_raw', 
                     folder_to_save = f'{NOVA_HOME}/tools/image_reports')